In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [6]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [7]:
df.sample(5)

,Question,Answer
13760,How did they manage to smoke?,"They threw a cigarette overboard, and made the..."
65054,What about this one for your bedroom?,ME: Hmmm no that one is also far too small
44698,What are Islamic extremists favourite drink?,Smirnoff Isis
42954,What do you call a white man surrounded by man...,A warden
1270,Where does a dog go when he loses his tail?,A retail store.


In [8]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [9]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,"Just give me a straight answer, Walter, cause ...",I suppose if you're looking for a male between...
1,Did you see a message to call me when you got ...,Tuesday. I had a late lunch.
2,"Only,good people love Linda, see. What Linda h...","I love Linda. Myself, I love Linda so much!"
3,"'But we're alone now, and I'm singing this son...","Your bone structure, shoulders, neck..."
4,What are sardines?,A little fish that smells like fingers.


In [10]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296389 entries, 0 to 296388
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Question  296389 non-null  object
 1   Answer    296389 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Tokenization

In [11]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [12]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs

Batchwise input and data preparation

In [13]:
import itertools

PAD_token = 0  # used for padding short sentences
SOS_token = 1  # start of sentence token
EOS_token = 2  # End-of-sentence token

def indexesFromSentence(voc , sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l , fillvalue = PAD_token):
    return list(itertools.zip_longest(*l , fillvalue = fillvalue))

def binaryMatrix(l , value=PAD_token):
    m = []
    for i , seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(False)
            else:
                m[i].append(True)
    return m

# returns padded input sequence tensor and lengths 
def inputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar , lengths

# returns padded target sequence tensor, padding mask , and max target length
def outputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar , mask , max_target_len

#returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        print("Input sentence-",pair[0])
        print("Output sentece-",pair[1])
        print("-----------------------")
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [14]:
import random

# Create a vocabulary instance
voc = Voc('test')

# Create pairs from your existing dataframe and normalize them
print("Creating pairs from dataframe...")
pairs = []
for _, row in dfn.iterrows():
    question = normalizeString(row['Question'])
    answer = normalizeString(row['Answer'])
    pairs.append([question, answer])

# Build vocabulary from pairs
print("Building vocabulary...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

# Example for validation
print("\nTesting with a small batch...")
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nBatch information:")
print("Input shape:", input_variable.shape)
print("Lengths:", lengths)
print("Output shape:", target_variable.shape)

Creating pairs from dataframe...
Building vocabulary...

Testing with a small batch...
Input sentence- what is the difference between a cock and a penis ?
Output sentece- a cock is an animal . penis is a male sexual organ .
-----------------------
Input sentence- how do you wake a sleeping lady gaga ?
Output sentece- you poke poke poker face
-----------------------
Input sentence- visions of utopia in nineteenth century american fiction .
Output sentece- what else . of course what else ? and what are you writing about for your thesis ?
-----------------------
Input sentence- oh yah . norm made some eggs .
Output sentece- you had breakfast yet margie ?
-----------------------
Input sentence- why do actors like snooker halls ?
Output sentece- because that is where they get their best cues .
-----------------------

Batch information:
Input shape: torch.Size([12, 5])
Lengths: tensor([12, 10, 10,  9,  8])
Output shape: torch.Size([19, 5])


In [15]:
def filterPair(voc, pair):
	# Return True if all words in both sentences are in vocab
	return all(word in voc.word2index for word in pair[0].split()) and \
		   all(word in voc.word2index for word in pair[1].split())

# Get valid pairs
valid_pairs = [pair for pair in pairs if filterPair(voc, pair)]

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(valid_pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nInput variable:")
print("input shape:", input_variable.shape)
print("lengths:", lengths)
print("\nTarget variable:")
print("Output shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len:", max_target_len)

Input sentence- easily . . . the question is can we save ourselves ? be a shame to win the battle only to lose the war .
Output sentece- can you save her ?
-----------------------
Input sentence- i do not think so .
Output sentece- look at me . i am pretty and sort of clever and very well intentioned and dear god i d tear your heart out !
-----------------------
Input sentence- marvin a . pickles .
Output sentece- state your name please .
-----------------------
Input sentence- what is that ?
Output sentece- thank you for your gift . i am sorry about marie .
-----------------------
Input sentence- nothing .
Output sentece- what ?
-----------------------

Input variable:
input shape: torch.Size([26, 5])
lengths: tensor([26,  7,  6,  5,  3])

Target variable:
Output shape: torch.Size([26, 5])
mask shape: torch.Size([26, 5])
max_target_len: 26


Architecture

In [ ]:
import torch.nn as nn
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.n_layers = n_layers
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout),
                          bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu())
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        # Sum bidirectional hidden states
        if self.gru.bidirectional:
            hidden = self._sum_bidirectional_hidden(hidden)
        return outputs, hidden

    def _sum_bidirectional_hidden(self, hidden):
        # hidden shape: (num_layers * num_directions, batch, hidden_size)
        # Sum bidirectional layers
        hidden = hidden.view(self.n_layers, 2, -1, self.hidden_size)
        hidden = torch.sum(hidden, dim=1)  # sum over directions
        return hidden

Attention layer and decoder

attention layer

In [17]:
import torch.nn.functional as F

# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
        
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)
        
    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
        
    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
        
        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
        
        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

decoder

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.attn = Attn(attn_model, hidden_size)
        
    def forward(self, input_step, last_hidden, encoder_outputs):
            embedded = self.embedding(input_step)
            embedded = self.embedding_dropout(embedded)
            rnn_output, hidden = self.gru(embedded, last_hidden)
            attn_weights = self.attn(rnn_output, encoder_outputs)
            context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
            rnn_output = rnn_output.squeeze(0)
            context = context.squeeze(1)
            concat_input = torch.cat((rnn_output, context), 1)
            concat_output = torch.tanh(self.concat(concat_input))
            output = self.out(concat_output)
            output = F.log_softmax(output, dim=1)  # Changed to log_softmax
            return output, hidden


Loss function

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.gather(inp, 1, target.view(-1, 1)).squeeze(1)
    loss = crossEntropy.masked_select(mask).mean()
    return loss, nTotal.item()

Training Model and Evaluation

In [ ]:
import os

# Model configuration
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set training device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)

# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN('dot', embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

# Use CUDA if available
encoder = encoder.to(device)
decoder = decoder.to(device)

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.9
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000
print_every = 100
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# Set model name and directory for saving
model_name = 'chatbot_model'
corpus_name = 'cornell_movie_dialogs'
save_dir = os.path.join("data", "save")

# Create save directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Set loadFilename to None as we're training from scratch
loadFilename = None



Building optimizers ...


In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
               embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
               print_every, save_every, clip, corpus_name, loadFilename):
    # Load batches for each iteration
    # Instead of random.choice for each batch, shuffle and batch the data properly
    # This is a simplified example; implement a proper batching mechanism
    training_batches = [batch2TrainData(voc, pairs[i:i+batch_size])
                        for i in range(0, len(pairs), batch_size)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    
    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                    decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(
                iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if iteration % save_every == 0:
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(
                encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder,
          embedding, encoder_optimizer, decoder_optimizer, batch_size, clip):
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # lengths = lengths.to(device) lengths are expected to be in cpu

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if teacher forcing should be used for this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    loss = 0
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_input = target_variable[t].view(1, -1)  # Next input is current target
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            _, topi = decoder_output.topk(1)  # [64, 1]
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss

    # Perform backpropagation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Starting Training!
Input sentence- for two whole weeks an there was not hardly nobody there to talk to but me . the manager of the place told me it was the off season an it must a ben because other n me there was just this year old kid delyon that did not appear to be all there an this old married pair next to me that was always hollerin for quiet . can you imagine ? all you could a heard there was a pin an them hollerin away . . .
Output sentece- you ve been staying in a motel all this time ?
-----------------------
Input sentence- you go fuck yourself . you go fuck yourself . you go fuck yourself . my love was hurt she got hurt . i am in love with her and you hurt you and you are gonna fucking get hurt . you do not touch her i love her .
Output sentece- i haven t even told you what is happened . your girl that you have that works there for you threatened me and two men just chased me extorted money dean go fuck yourself that shit does not have anything to do with me this is a legitim

evaluation 

In [27]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    
    # Add debug prints
    print("Tokens:", tokens)
    print("Token values:", [token.item() for token in tokens])
    
    # indexes -> words
    decoded_words = []
    for token in tokens:
        token_idx = token.item()
        if token_idx in voc.index2word:
            decoded_words.append(voc.index2word[token_idx])
        else:
            # If the index is not in the vocabulary, add a placeholder
            decoded_words.append(f"<UNKNOWN:{token_idx}>")
    
    return decoded_words


# Greedy decoding
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = decoder_input.view(1, 1)
            
            # Stop if we predict EOS_token
            if decoder_input.item() == EOS_token:
                break
            
        return all_tokens, all_scores

def normalize_string(s):
    # Same normalization function you used during training
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize searcher module
searcher = GreedySearchDecoder(encoder, decoder)

def chat_with_bot():
    print("Chat with your bot! (type 'quit' to exit)")
    while True:
        # Get input sentence
        input_sentence = input('You: ')
        if input_sentence.lower() == 'quit':
            break
            
        # Normalize sentence
        normalized = normalize_string(input_sentence)
        print(f"Normalized input: '{normalized}'")
        
        # Check if all words are in vocabulary
        unknown_words = [word for word in normalized.split() if word not in voc.word2index]
        if unknown_words:
            print(f"Warning: These words are not in the vocabulary: {unknown_words}")
        
        try:
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, normalized)
            
            # Remove EOS token and PAD tokens
            output_words = [word for word in output_words if word not in ['EOS', 'PAD']]
            
            # Print response
            print('Bot:', ' '.join(output_words))
            
        except Exception as e:
            print(f"Error during evaluation: {str(e)}")
            import traceback
            traceback.print_exc()

In [28]:
chat_with_bot()

Chat with your bot! (type 'quit' to exit)
Normalized input: 'hello'
Shape after embedding: torch.Size([2, 1, 500])
Shape after packing: torch.Size([2, 500])
Shape after GRU: torch.Size([2, 1000])
Shape after unpacking: torch.Size([2, 1, 1000])
Shape after summing bidirectional outputs: torch.Size([2, 1, 500])
Tokens: tensor([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
       device='cuda:0')
Token values: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
Bot: . . . . . . . . . . . . . . .
Normalized input: 'again the same output'
Shape after embedding: torch.Size([5, 1, 500])
Shape after packing: torch.Size([5, 500])
Shape after GRU: torch.Size([5, 1000])
Shape after unpacking: torch.Size([5, 1, 1000])
Shape after summing bidirectional outputs: torch.Size([5, 1, 500])
Tokens: tensor([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
       device='cuda:0')
Token values: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
Bot: . . . . . . 

In [30]:
def test_chatbot():
    print("Testing chatbot with direct debug output...")
    
    # Set models to evaluation mode
    encoder.eval()
    decoder.eval()
    
    # Test sentence
    test_sentence = "hello how are you"
    print(f"Test sentence: '{test_sentence}'")
    
    # Normalize sentence
    normalized = normalize_string(test_sentence)
    print(f"Normalized input: '{normalized}'")
    
    try:
        # Convert to indexes
        indexes = [voc.word2index.get(word, UNK_token) for word in normalized.split()]
        print(f"Word indexes: {indexes}")
        
        # Add EOS token
        indexes.append(EOS_token)
        print(f"Word indexes with EOS: {indexes}")
        
        # Create tensor
        input_tensor = torch.LongTensor([indexes]).transpose(0, 1)
        print(f"Input tensor shape: {input_tensor.shape}")
        input_tensor = input_tensor.to(device)
        
        # Create lengths tensor
        lengths = torch.tensor([len(indexes)])
        print(f"Lengths tensor: {lengths}")
        
        # Encoder forward pass
        with torch.no_grad():
            encoder_outputs, encoder_hidden = encoder(input_tensor, lengths.cpu())
            print(f"Encoder outputs shape: {encoder_outputs.shape}")
            print(f"Encoder hidden shape: {encoder_hidden.shape}")
        
        # Prepare decoder input and hidden state
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        print(f"Decoder hidden shape: {decoder_hidden.shape}")
        
        # Initial decoder input is SOS token
        decoder_input = torch.LongTensor([[SOS_token]]).to(device)
        print(f"Initial decoder input: {decoder_input}")
        
        # Decode one word at a time
        output_words = []
        max_length = 20  # Maximum response length
        
        for _ in range(max_length):
            with torch.no_grad():
                # Ensure decoder_input has the right shape
                # It should be [1, batch_size] where batch_size is 1 in this case
                decoder_input = decoder_input.view(1, -1)
                print(f"Decoder input shape: {decoder_input.shape}")
                
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )
                print(f"Decoder output shape: {decoder_output.shape}")
                
                # Get most likely word index
                _, topi = decoder_output.topk(1)
                print(f"Top token index: {topi.item()}")
                
                # If EOS, stop decoding
                if topi.item() == EOS_token:
                    output_words.append('EOS')
                    break
                
                # Add the decoded word to output
                if topi.item() in voc.index2word:
                    decoded_word = voc.index2word[topi.item()]
                    print(f"Decoded word: '{decoded_word}'")
                    output_words.append(decoded_word)
                else:
                    print(f"Unknown index: {topi.item()}")
                    output_words.append(f"<UNK:{topi.item()}>")
                
                # Next input is the decoded word
                decoder_input = torch.LongTensor([[topi.item()]]).to(device)
        
        # Output final response
        print("\nFinal bot response:")
        print(' '.join(output_words))
        
    except Exception as e:
        print(f"Error during testing: {str(e)}")
        import traceback
        traceback.print_exc()

# Call the test function
test_chatbot()

Testing chatbot with direct debug output...
Test sentence: 'hello how are you'
Normalized input: 'hello how are you'
Word indexes: [1211, 214, 83, 22]
Word indexes with EOS: [1211, 214, 83, 22, 2]
Input tensor shape: torch.Size([5, 1])
Lengths tensor: tensor([5])
Shape after embedding: torch.Size([5, 1, 500])
Shape after packing: torch.Size([5, 500])
Shape after GRU: torch.Size([5, 1000])
Shape after unpacking: torch.Size([5, 1, 1000])
Shape after summing bidirectional outputs: torch.Size([5, 1, 500])
Encoder outputs shape: torch.Size([5, 1, 500])
Encoder hidden shape: torch.Size([4, 1, 500])
Decoder hidden shape: torch.Size([2, 1, 500])
Initial decoder input: tensor([[1]], device='cuda:0')
Decoder input shape: torch.Size([1, 1])
Decoder output shape: torch.Size([1, 67171])
Top token index: 18
Decoded word: '.'
Decoder input shape: torch.Size([1, 1])
Decoder output shape: torch.Size([1, 67171])
Top token index: 18
Decoded word: '.'
Decoder input shape: torch.Size([1, 1])
Decoder output